# Naive Bayes

# A1 Transmitter 
A "1" is observed at the local end of an error-prone binary communication channel Y. The source at the far end of the channel is known "a priori" to transmit ones with probability p(X=1)=0.20. The channel is defined by the transmission matrix of conditional probabilities 

$$
p(Y=1|X=1)=0.90 \\ p(Y=1|X=0)=0.25\\
p(Y=0|X=1)=0.10 \\ p(Y=0|X=0)=0.75
$$


completely characterized. What symbol X*=? did the source most likely send? Calculate using Bayes' theorem.

# Lösungsweg:
$X=0:\quad     p(X=0|Y=1) = p(Y=1|X=0)\cdot p(X=0)\cdot c = 0.25 \cdot  0.8 \cdot  c = 0.2\cdot  c$


$X=1:\quad    p(X=1|Y=1) = p(Y=1|X=1)\cdot p(X=1)\cdot c = 0.9 \cdot  0.2 \cdot  c = 0.9 \cdot  0.2 \cdot  c$

Solution: X* = 0 because p(X=0|Y=1) die maximale Wk. ist.

Note: constant $c = 1/p(Y=1)$ and is the same in both cases, which is why we can neglect its exact value. You also can marginalize over X to get $P(Y)$.

# A2 - Sentence Segmenter: 
An unknown (sentence) segmenter operating on heuristic principles considers a corpus as a sequence of word forms. A point is interpreted as a sentence boundary for segmentation, depending on several things:

    - Whether the preceding word is an abbreviation,
    - if so, whether it is always terminated by a dot, and whether it is clustered at the end of the sentence
    - how long the previous segment is, etc. ...
    
It is known "a priori" that the segmenter S works correctly in 99% of all cases, incorrectly otherwise. After segmentation, the corpus is classified by language. The correctness k of the classification result C would depend on a correct sentence segmentation:

$p(C=k|S=k)=0.80 $<br>$p(C=k|S=f)=0.30$

$p(C=f|S=k)=0.20 $<br>$p(C=f|S=f)=0.70$

We observe a corpus that has been assigned an incorrect language, but we know that overall the classifier is wrong only 14% of the time: p(C=f)=0.14. What is the probability that incorrect segmentation has occurred? (Hint: Bayes' theorem)

### Lösung:
$p(S=f|C=f) = \frac{p(C=f|S=f)\cdot p(S=f)}{p(C=f)} =  .70 \cdot .01 / .14  = 70/14 /100 = 10/2 /100 = 5\%$

# A3 - Sniffer Dog
The sniffer dog problem: A customs dog should bark whenever it smells a passenger transporting drugs. Let $B$ be the event "dog barks" and $D$ be the event "passenger is carrying drugs." The following probabilities are given: $P(B \mid D) = 0.98$, $P( B \mid \overline D) = 0.03$, $P(D) = 0.01$.

Determine the probability that a passenger is really carrying drugs if the dog barks, and the probability that the passenger is not carrying drugs if the dog does not bark.

### Solution

We are looking for $P(D\mid B)$ and $P(\overline D\mid \overline B)$. From the given probabilities:

$P(\overline B\mid D)=0.02$, $P(\overline B\mid \overline D)=0.97$ and $P(\overline D)=0.99$. Using Bayes' law:
$$ \begin{array}{cc}
P(D\mid B) &= \displaystyle \frac{P(D) P(B\mid D)}{P(D) P(B\mid D)+P(\overline D) P(B\mid \overline D)} &= 0.248 \\[4ex]
P(\overline D\mid \overline B) &= \displaystyle \frac{P(\overline D) P(\overline B\mid \overline D)}{P(\overline D) P(\overline B\mid \overline D)+P(D) P(\overline B\mid D)} &= 1.000
\end{array} $$

# P1 Language Detection

You can extend the simple count based language detection module from an earlier exercise to use Naive Bayes Principles. Instead of only looking for hits in the top ngrams of a language, we can consider probabilities of ngrams. 

For simplicity (naivety), we assume that all ngram probabilities only depend on the language, but are otherwise independent of each other.

For any ngram $x_i$ and language $L_k$ the model calculates the probability distributions $p(x_i | L_k)$ and $p(x_i)$. The final evaluation if a class is matching a certain document is made by
$$ \hat{y} = \text{argmax}_{k∈{1,...,K}} p(L_k)  \prod^n_{i=1} p(x_i| L_k) $$


We will also use “Add one” or “Laplacian” Smoothing. For the observed ngrams a pseudo count is
introduced and a virtual count of 1 is added.


 $$p(x_i| L_k) = \frac{count(x_i| L_k) + 1}{ count(L_k) + |V|}$$

For unknown ngrams in the documents we have to predict, we could also introduce a pseudo count
and 0-probabilities are prevented.
 $$p(xi| L_k) = \frac{1}{count(L_k) + |V|}$$

In [1]:
import nltk
import pathlib
import matplotlib.pyplot as plt
from functools import reduce
english_text = nltk.corpus.gutenberg.raw(nltk.corpus.gutenberg.fileids())[:200000]
def load_german_text(path):
    text = ""
    for f in pathlib.Path(path).glob("*.txt"):
        with open(f, "r") as openf:
            text += openf.read()
    return text
german_text = load_german_text("../../Woche 5/tagesschau_corpus")[:200000]           

In [2]:
import string
import re
def clean(s: str) -> str:
    valid=string.ascii_letters+string.digits+" "
    s = s.lower()
    for c in valid:
        s = "".join([c if c in valid else " " for c in s ])
    s = re.sub("[\s]+", " ", s)
    return s

In [3]:
n=3
topk=1000
texts = {"english":english_text, "german":german_text}

def create_ngrams(texts: dict):
    return ["".join(g) for w in nltk.tokenize.word_tokenize(texts) for g in nltk.ngrams(w, n)]

counts = {k:nltk.FreqDist(create_ngrams(v)) for k, v in texts.items()}


In [4]:
print(counts["german"].most_common(5))
print(counts["english"].most_common(5))

vocab_ger = set([k for k,v in counts["german"].most_common(topk)])
vocab_eng = set([k for k,v in counts["english"].most_common(topk)])
vocab_size = len(vocab_ger.union(vocab_eng))
print(vocab_size)


[('ich', 1307), ('der', 1233), ('sch', 1215), ('ein', 1118), ('che', 1066)]
[('the', 1772), ('and', 1293), ('her', 1102), ('ing', 1096), ('ver', 732)]
1637


In [5]:
ngram_language_prob = {k:{k2:(v2+1)/(sum(v.values())+vocab_size) for k2,v2 in v.most_common(topk)} for k,v in counts.items()}
v0 = list(counts.values())[0]
print(v0.most_common(5))
print(sum(v0.values()))
print((1772+1)/(83568+vocab_size))
list(ngram_language_prob["english"].items())[:5]


[('the', 1772), ('and', 1293), ('her', 1102), ('ing', 1096), ('ver', 732)]
83568
0.020808637990728244


[('the', 0.020808637990728244),
 ('and', 0.015186902177102283),
 ('her', 0.012945249691919488),
 ('ing', 0.012874831289243589),
 ('ver', 0.008602781526905698)]

In [6]:
test = "Das ist ein deutscher Satz."
d = create_ngrams(clean(test))

p =  {language: [probs.get(ngram,1/(len(probs)+vocab_size)) for ngram in d] for language, probs in ngram_language_prob.items()}
probs_en = list(ngram_language_prob.values())[0]
print(d)
print(len(probs_en))

print("English:")
print(p["english"])
print("German:")
print(p["german"])


['das', 'ist', 'ein', 'deu', 'eut', 'uts', 'tsc', 'sch', 'che', 'her', 'sat', 'atz']
1000
English:
[0.0003792188092529389, 0.0012205856463822545, 0.001185376445044305, 0.0003792188092529389, 0.0003792188092529389, 0.0003792188092529389, 0.0003792188092529389, 0.0003792188092529389, 0.0006337656240830937, 0.012945249691919488, 0.000539874420515228, 0.0003792188092529389]
German:
[0.00354588142361873, 0.0034668890156668276, 0.009821389388686532, 0.000386185105542634, 0.001386755606266731, 0.0007635932768683899, 0.001597402027471804, 0.010672752007723703, 0.009364988809408873, 0.0024575415807258525, 0.00026330802650634136, 0.0002545310922894633]


In [7]:
 p2 = {lang: reduce(lambda x, y: x*y, probs) for lang, probs in p.items()}

In [8]:
p2

{'english': 7.227456801133564e-39, 'german': 1.2983696335931752e-33}

In [9]:
prior = [1/len(ngram_language_prob)] * len(ngram_language_prob)
prior

[0.5, 0.5]

In [10]:
posterior = {lang: probs*pr for  (lang,probs), pr in zip(p2.items(), prior)}
sorted_counts = sorted(list(posterior.items()), key=lambda x: -x[1])
print(posterior)
print(sorted_counts)

{'english': 3.613728400566782e-39, 'german': 6.491848167965876e-34}
[('german', 6.491848167965876e-34), ('english', 3.613728400566782e-39)]


In [11]:
class NGramLanguageClassifierBayes:
    def __init__(self,texts, n=3, topk=1000):
        self.n = n
        counts = {k:nltk.FreqDist(self.create_ngrams(v)) for k, v in texts.items()}
        vocab = [[k for k,v in counts_country.most_common(topk)] for lang, counts_country in counts.items()]
        vocab_size = len(set(sum(vocab, [])))
        self.vocab_size= vocab_size
        self.ngram_language_prob = {k:{k2:(v2+1)/(sum(v.values())+vocab_size) for k2,v2 in v.most_common(topk)} for k,v in counts.items()}
            
        
    def create_ngrams(self, texts: dict):
        return ["".join(g) for w in nltk.tokenize.word_tokenize(texts) for g in nltk.ngrams(w, self.n)]
    
    def classify(self,text, prior = None):
        d =  self.create_ngrams(clean(text))
        p =  {language: [probs.get(ngram,1/(len(probs)+self.vocab_size)) for ngram in d] for language, probs in self.ngram_language_prob.items()}
        p = {lang: reduce(lambda x, y: x*y, probs) for lang, probs in p.items()}
        
        if prior is None:
            prior = [1/len(self.ngram_language_prob)] * len(self.ngram_language_prob)
        
        # Sort list by frequency
        p = {lang: probs*pr for  (lang,probs), pr in zip(p.items(), prior)}
        
        sorted_counts = sorted(list(p.items()), key=lambda x: -x[1])

        return sorted_counts
    

In [12]:
NGLC = NGramLanguageClassifierBayes({"english":english_text, "german":german_text}, n=3)  

NGLC.classify("Das ist ein deutscher Satz.")


[('german', 6.491848167965876e-34), ('english', 3.613728400566782e-39)]

In [13]:
NGLC.classify("Arnold Schwarzenegger is a great actor from Österreich.")

[('german', 3.1990309589622516e-98), ('english', 2.8758043057208075e-100)]

In [14]:
NGLC.classify("I am pretty sure Arnold Schwarzenegger is a great actor from Österreich.")

[('english', 1.4845823530782987e-117), ('german', 7.556673099513004e-119)]